In [1]:
'''
author: Felix Hol
date: 2020 Nov 25
content: code to calculate distance of nearest mosquito and identify its particle number
'''

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import itertools as it
import pandas as pd
import pims
import skimage
from skimage import data, io, util
from skimage.measure import label, regionprops
from skimage.morphology import binary_dilation, erosion, dilation, opening, binary_closing, closing, white_tophat, remove_small_objects, disk, black_tophat, skeletonize, convex_hull_image
import scipy
import trackpy as tp
import pylab
import math
from joblib import Parallel, delayed
import multiprocessing
from datetime import datetime
from tqdm import tnrange, tqdm
import pickle
import glob
import cv2 as cv

In [5]:
#### set directories where to get images and where to store output, and specifics of experiment
dataDir = '/Users/felix/Documents/mosquitoes/mosquitoData/P3/all/'
saveDir = '/Users/felix/Documents/mosquitoes/mosquitoData/P3/all/'

In [6]:
mosDataName = 'aeg_ctrl1_2_1105'
trackPickleName = saveDir + mosDataName + '_tracks.pkl'
tFull = pd.read_pickle(trackPickleName)

In [7]:
### create a new dataframe based on tracking dataframe having nearest neighbor distances for all particles. 

T = tFull.copy()
T['nnDist'] = np.nan
T['nnP'] = np.nan

for f in T.frame.unique():
    t = T.loc[T.frame == f]
    if len(t) > 1:
        for p in t.particle.unique():
            xp = t.loc[t.particle == p].x.values
            yp = t.loc[t.particle == p].y.values
            dist = []
            others = np.delete(t.particle.unique(), np.where(t.particle.unique() == p))
            for o in others:
                xo = t.loc[t.particle == o].x.values
                yo = t.loc[t.particle == o].y.values
                D = np.sqrt(np.abs(xp - xo)**2 + np.abs(yp - yo)**2)
                dist = np.append(dist, D)
            T.loc[T.loc[(T.frame == f) & (T.particle == p)].index, 'nnDist'] = np.min(dist)
            T.loc[T.loc[(T.frame == f) & (T.particle == p)].index, 'nnP'] = int(others[np.argmin(dist)])

In [9]:
### remove all particles that do not have a nearest neighbor, i.e. remove all frames with only 1 mosquito

P = T.copy()
P = P[~P.nnDist.isna()]
P.head()

,y,x,frame,particle,species,age,nnDist,nnP
121,94.734834,2547.198630,920,3,aeg,21,2104.634539,2.0
122,1739.600511,1234.223595,920,2,aeg,21,2104.634539,3.0
123,99.075013,2542.261769,921,3,aeg,21,2099.001525,2.0
124,1739.964822,1233.335478,921,2,aeg,21,2099.001525,3.0
125,98.934899,2536.698506,922,3,aeg,21,2095.467539,2.0


In [15]:
'''create new dataframe NN listing average nearest neighbor distance per frame (nnDo), 
the expected NN distance for N particles (nnDe), and the ratio nnI (i.e. NN index)
'''


NN = pd.DataFrame(columns=['frame', 'N', 'nnDo', 'nnDe'])
areaWidth = 2806
areaHigth = 2736
Area = areaWidth * areaHigth ### in pix, check what actualy numbers should be (e.g. ~1893 * 1911 for invitation)
perimeter = 2 * areaHigth + 2 * areaWidth ### in pix, check what actualy numbers should be (~7608 for invitation)

for f in P.frame.unique():
    pP = P.loc[P.frame == f]
    N = len(pP)
    nnDo = np.mean(pP.nnDist)
    nnDe = 1/2 * np.sqrt(Area / N) + (0.0514 * perimeter) / N + (0.041 * perimeter) / (N * np.sqrt(N))
    NN = NN.append({'frame': f, 'N': N, 'nnDo': nnDo, 'nnDe': nnDe}, ignore_index = True)
    
NN['nnI'] = NN.nnDo / NN.nnDe  
NN['Z'] = (NN.nnDo - NN.nnDe) / (0.26136 / (np.sqrt((NN.N ** 2) / Area)))
NN['Zabs'] = np.abs(NN.Z)

In [16]:
NN.head()

,frame,N,nnDo,nnDe,nnI
0,920.0,2.0,2104.634539,1425.14731,1.476784
1,921.0,2.0,2099.001525,1425.14731,1.472831
2,922.0,2.0,2095.467539,1425.14731,1.470352
3,923.0,2.0,2092.696984,1425.14731,1.468407
4,924.0,2.0,2086.685982,1425.14731,1.464190
